# Method - 1 Supervised Model Development

In [ ]:
# Libraries
from utils import read_data, rRMSE_per_case #get_batch, fit_biExponential_model_signal
import numpy as np
import matplotlib.pyplot as plt
import os
from torch import nn
import torch
from torch import optim
from tqdm.notebook import tqdm

In [ ]:
# Addresses
file_dir='../public_training_data/'
fname_gt ='_IVIMParam.npy'
fname_gtDWI ='_gtDWIs.npy'
fname_tissue ='_TissueType.npy'
fname_noisyDWIk = '_NoisyDWIk.npy'
file_Resultdir='../Result/'

b_values = [0, 5, 50, 100, 200, 500, 800, 1000]

In [ ]:
# Save NLLS performance
tumor, non_tumor = 0, 0
ctr = 0
for image_number in tqdm(range(950, 1000)):
    nlls = np.load(os.path.join(file_Resultdir, f'{(image_number + 1):04d}.npy'))
    params = read_data(file_dir, fname_gt, image_number + 1)
    tissue = read_data(file_dir, fname_tissue, image_number + 1)
    t, nt = rRMSE_per_case(nlls[:,:,0], nlls[:,:,1], nlls[:,:,2], params[:,:,0], params[:,:,1], params[:,:,2], tissue)
    tumor += t
    non_tumor += nt
    ctr += 1
print(tumor/ctr, non_tumor/ctr)

In [ ]:
# Define the Model
class PIA(nn.Module):
    
    def __init__(self, b_values = [0, 5, 50, 100, 200, 500, 800, 1000],
                hidden_dims= [32, 64, 128, 256, 512],
                predictor_depth=2):
         super(PIA, self).__init__()

         self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
         self.number_of_signals = len(b_values)
         self.sigmoid = nn.Sigmoid()
         modules = []
         in_channels = self.number_of_signals
         for h_dim in hidden_dims:
             modules.append(nn.Sequential(
                 nn.Linear(in_features=in_channels, out_features=h_dim),
                 nn.LeakyReLU()))
             in_channels = h_dim
         self.encoder = nn.Sequential(*modules).to(self.device)
         D_predictor = []
         for _ in range(predictor_depth):
             D_predictor.append(nn.Sequential(
                 nn.Linear(in_features=hidden_dims[-1], out_features=hidden_dims[-1]),
                 nn.LeakyReLU())
                 )
         D_predictor.append(nn.Linear(hidden_dims[-1], 1))
         self.D_predictor = nn.Sequential(*D_predictor).to(self.device)

         D_star_predictor = []
         for _ in range(predictor_depth):
             D_star_predictor.append(nn.Sequential(
                 nn.Linear(in_features=hidden_dims[-1], out_features=hidden_dims[-1]),
                 nn.LeakyReLU())
                 )
         D_star_predictor.append(nn.Linear(hidden_dims[-1], 1))
         self.D_star_predictor = nn.Sequential(*D_star_predictor).to(self.device)
         f_predictor = []
         for _ in range(predictor_depth):
             f_predictor.append(nn.Sequential(
                 nn.Linear(in_features=hidden_dims[-1], out_features=hidden_dims[-1]),
                 nn.LeakyReLU())
                 )
         f_predictor.append(nn.Linear(hidden_dims[-1], 1))
         self.f_predictor = nn.Sequential(*f_predictor).to(self.device)


    def encode(self, x):
        """
        Encodes the input by passing through the encoder network
        and returns the latent codes.
        :param input: (Tensor) Input tensor to encoder
        :return: (Tensor) List of latent codes
        """
        result = self.encoder(x)   
        D = self.D_predictor(result)
        D_star = 5 + self.D_star_predictor(result)
        f = self.sigmoid(self.f_predictor(result))      
        return f, D, D_star


In [ ]:
# Observe the inherent noise std for different b-values
for image_number in range(1):
    params = read_data(file_dir, fname_gt, image_number + 1)
    clean = read_data(file_dir, fname_gtDWI, image_number + 1)
    tissue = read_data(file_dir, fname_tissue, image_number + 1)
    coordBody = np.argwhere(tissue != 1)
    k = read_data(file_dir, fname_noisyDWIk, image_number + 1)
    noisy = np.abs(np.fft.ifft2(k, axes=(0,1) ,norm='ortho'))
    clean = np.abs(clean)
    noisy = noisy[[coordBody[:,0], coordBody[:,1]]]
    clean = clean[[coordBody[:,0], coordBody[:,1]]]
    noisy = noisy/clean[:, 0, np.newaxis]
    clean = clean/clean[:, 0, np.newaxis]
    noise = noisy - clean
    for i in range(8):
        plt.figure()
        a = plt.hist(noise[:,i], bins=100)
        plt.title(f'{np.std(noise[:, i])}')


In [ ]:
model = PIA()
params = model.parameters()
lr = 3e-4
optimizer = optim.Adam(params, lr=lr)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
loss_fcn = nn.MSELoss()
model = model.train()

In [ ]:
for ep in range(500):
    for image_number in range(950):
        params = read_data(file_dir, fname_gt, image_number + 1)
        tissue = read_data(file_dir, fname_tissue, image_number + 1)
        coordBody = np.argwhere(tissue != 1)
        k = read_data(file_dir, fname_noisyDWIk, image_number + 1)
        noisy = np.abs(np.fft.ifft2(k, axes=(0,1) ,norm='ortho'))

        for pix in tqdm(range(250, coordBody.shape[0])):
            ix, jx = coordBody[pix, 0], coordBody[pix, 1]
            f_true, D_true, D_star_true = torch.from_numpy(params[ix, jx, :]).to(device).float()
            D_true *= 1000
            D_star_true *=1000
            signal = noisy[ix, jx, :]/noisy[ix, jx, 0]
            signal = torch.from_numpy(signal).to(device).float()
            f, D, D_star = model.encode(signal)
            loss = loss_fcn(D, D_true) + loss_fcn(f, f_true) + loss_fcn(D_star, D_star_true)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        tumor, non_tumor = 0, 0
        ctr = 0
        print('Testing')
        for image_number in tqdm(range(950, 1000)):
            params = read_data(file_dir, fname_gt, image_number + 1)
            tissue = read_data(file_dir, fname_tissue, image_number + 1)
            k = read_data(file_dir, fname_noisyDWIk, image_number + 1)
            noisy = np.abs(np.fft.ifft2(k, axes=(0,1) ,norm='ortho'))
            samples = noisy.reshape(-1,8)
            samples = torch.from_numpy(samples/samples[:, 0, np.newaxis]).to(device).float()
            f, D, D_star = model.encode(samples)
            f = f.detach().cpu().numpy().reshape(200,200)
            D = D.detach().cpu().numpy().reshape(200,200)/1000
            D_star = D_star.detach().cpu().numpy().reshape(200,200)/1000
            t, nt = rRMSE_per_case(f, D, D_star, params[:,:,0], params[:,:,1], params[:,:,2], tissue)
            tumor += t
            non_tumor += nt
            ctr += 1
        print(tumor/ctr, non_tumor/ctr)
        PATH = f'../ivim_models/pia_model_{tumor/ctr:.2f}.pt'
        torch.save(model, PATH)

In [ ]:

fig, ax = plt.subplots(3,1, figsize=(15,45))
title = ['f ', 'D ', 'D_star']
ylims = [(0.0, 1.0), (0.0, 0.003), (0.003, 0.3)]
for c in range(3):
    y = nlls[:, :, c].flatten() 
    x = params[:, :, c].flatten()
    #nbins=30
    #k = gaussian_kde([x,y])
    #xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
    #zi = k(np.vstack([xi.flatten(), yi.flatten()]))
    #ax[c].pcolormesh(xi, yi, zi.reshape(xi.shape), cmap="hot", shading='auto')
    ax[c].scatter(x,y, color='b', s=4, alpha=0.5)

    err = np.mean(np.abs(x - y))
    corr = np.corrcoef(x,y)[0,1]
    ax[c].xaxis.set_tick_params(labelsize=20)
    ax[c].yaxis.set_tick_params(labelsize=20)
    #ax[r,c].set_title(fr'{title[c]}, MAE = {err:.3f}, $\rho$ = {corr:.3f}')
    ax[c].set_xlabel('true', fontsize=24)
    ax[c].set_ylabel('predicted', fontsize=24)